# This Notebook gives an example for using the implemented quantum circuit
In this example we will train the network on the MNIST data set for binary classification.
The task is to provide a classifier for numbers *0* and *4*.
We will train the network using the default parameters and evaulate the result.

## Imports
In the beginning we start with importing data_utils to create our train and test sets as well as the network itself.

In [ ]:
from quantum.utils import data_utils
from quantum.network import QuantumNetwork

## Data Set 
Now we can create the data set. We have to decide which dimensionality we will tackle. Increasing the size increases the amount ouf time we have to spent.
Using *Dimension=4* we have 16 pixels to consider. Since we want a binary calssifier we have to filter the values in the dataset, contradicting exampkles will also be removed.
We also have to define a mapping of Labesl to quantum states like `{label: 'state'}`.

In [ ]:
DIMENSION = 4
LABELS = {0: '1', 4: '0'}
(X_TRAIN, Y_TRAIN), (X_TEST, Y_TEST) = data_utils.generate_dataset(DIMENSION, filter_values=True, value_true=0, value_false=4)

## Network
In the next step we initialize the network. Important parameters are the dimensionality which influences the circuit and the labels.
Hyperparameters need to be set using the respective function. Default values are provided by the paper. If we want to use the 
efficient circuit (recommended) set `efficient=True` in the initialization. 

In [ ]:
NETWORK = QuantumNetwork(DIMENSION, LABELS, shots=512, efficient=True)
NETWORK.set_spsa_hyperparameters() # use this function to controll the SPSA algorithm

## Training
FInally we can start training. This is straight forward but time-consuming.
For now we set a small number of epochs.

In [ ]:
NETWORK.train_epochs(X_TRAIN, Y_TRAIN, epochs=2)

## Testing
After training we can evaluate our network. Given are **Accuracy** and **Loss per Batch**.
*Test Accuracy* can be calculated manually. The weights are

In [ ]:
NETWORK.print_stats()
test_count = 0
for sample, label in zip(X_TEST, Y_TEST):
    if (NETWORK.predict(sample) == label):
        test_count += 1
print(f'Test Accuracy: {test_count/X_TEST.shape[0]}')

## Loading and Saving
The model parameters can be stored and saved using the pickle format in python.

In [ ]:
filename = 'my_model.pickle'
NETWORK.save_model(filename)
NETWORK = QuantumNetwork.load_model(filename)